In [ ]:
# Source code to obtain more than 1M rows by tiles blocks using pyvo in an authenticated service

In [1]:
# To avoid warnings

import warnings
warnings.simplefilter("ignore")

In [2]:
import pyvo.dal
from pyvo.auth import authsession, securitymethods
import requests
import getpass

url_tap_service = 'https://archive.cefca.es/catalogues/vo/tap/jplus-dr3'
archive_login_url = 'https://archive.cefca.es/catalogues/login'

user = input("Username:")
pwd = getpass.getpass("Password:")


# Creation of credentials structures
login_args = {'login': user, 'password': pwd, 'submit': 'Sign+In'}
login_header = {'Content-type': 'application/x-www-form-urlencoded', 'Accept': 'text/plain'}

pyvo.dal.tap.s = requests.Session()
response = pyvo.dal.tap.s.post(archive_login_url, data=login_args, headers=login_header)
response.raise_for_status()
auth = authsession.AuthSession()
auth.credentials.set(securitymethods.ANONYMOUS, pyvo.dal.tap.s) 
    

Username:tcivera
Password:········


In [3]:
# Obtain all the reference tiles in the DR and the blocks of tiles

num_tiles_per_block = 50 # Change this number depending the number of objects you expect by tile

service = pyvo.dal.TAPService(url_tap_service, auth)

resultset = service.search("SELECT tile_id FROM TileImage WHERE filter_id = jplus::rSDSS")

ref_tiles = resultset["tile_id"]

blocks = int(len(ref_tiles) / num_tiles_per_block)
if len(ref_tiles) % num_tiles_per_block > 0:
    blocks += 1
print("Total reference tiles:", len(ref_tiles), "Blocks:", blocks)

Total reference tiles: 1642 Blocks: 33


In [4]:
# Your query
sql_base = "SELECT obj.TILE_ID, obj.number, obj.ALPHA_J2000 AS RA ,obj.DELTA_J2000 AS DEC,obj.FLUX_AUTO," \
            "obj.FLUX_RELERR_AUTO,obj.class_star, lephare.* " \
            "FROM jplus.FNuDualObj obj, j{plus.PhotoZLephare lephare " \
            "WHERE obj.TILE_ID = lephare.TILE_ID AND obj.NUMBER = lephare.NUMBER AND lephare.photoz<0.35 AND lephare.photoz>0.05 AND obj.tile_id IN ({0})"
file_dest_name = 'block{0}.fits'

# Obtains your data in a fits file
for i in range(blocks):
    grp = list(ref_tiles[i * num_tiles_per_block:(i+1) * num_tiles_per_block])
    adql = sql_base.format(','.join([str(x) for x in grp]))
    job = service.submit_job(adql, format="FITS")
    dest_name = file_dest_name.format(i)
    try:
        job.run()
        job.wait()
        job.raise_if_error()
        with open(dest_name, "wb") as dest:        
            session = requests.Session()
            response = session.get(job.result_uri, auth=(user, pwd), verify=False)
            response.raise_for_status()
            dest.write(response.content)
    finally:
        job.delete()
    print("Done block", i)
    if i > 1:
        break
        
print("All done.")

/home/CEFCA/tcivera/miniconda3/envs/pyvoastroquery_env/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.cefca.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Done block 0


/home/CEFCA/tcivera/miniconda3/envs/pyvoastroquery_env/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.cefca.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Done block 1


/home/CEFCA/tcivera/miniconda3/envs/pyvoastroquery_env/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.cefca.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Done block 2
All done.
